In [2]:
# 기본적인 데이터 핸들링
import pandas as pd
import numpy as np
from datetime import datetime

# Crawling
import requests
from bs4 import BeautifulSoup

# 시간 제어 및 진행 경과 확인
from tqdm.notebook import tqdm
import time
import random

In [3]:
# DataFrame Load
df_semiconductor = pd.read_csv('Title_Link_Date/df_semiconductor.csv')
df_electric_car_battery = pd.read_csv('Title_Link_Date/df_electric_car_battery.csv')
df_petrochemistry = pd.read_csv('Title_Link_Date/df_petrochemistry.csv')
df_shipbuilding_shiping = pd.read_csv('Title_Link_Date/df_shipbuilding_shiping.csv')
df_steel = pd.read_csv('Title_Link_Date/df_steel.csv')

In [4]:
# Date열의 자료형 변환
df_semiconductor['Date'] = df_semiconductor['Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d').date())
df_electric_car_battery['Date'] = df_electric_car_battery['Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d').date())
df_petrochemistry['Date'] = df_petrochemistry['Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d').date())
df_shipbuilding_shiping['Date'] = df_shipbuilding_shiping['Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d').date())
df_steel['Date'] = df_steel['Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d').date())

In [5]:
# 2023년 기사만 사용
base_date = datetime.strptime("2022.12.31","%Y.%m.%d").date()

# 데이터프레임에서 올해 데이터만 사용하기 위해서 query문 사용
df_semiconductor = df_semiconductor.query('Date > @base_date')
df_electric_car_battery = df_electric_car_battery.query('Date > @base_date')
df_petrochemistry = df_petrochemistry.query('Date > @base_date')
df_shipbuilding_shiping = df_shipbuilding_shiping.query('Date > @base_date')
df_steel = df_steel.query('Date > @base_date')

In [6]:
# 기사 링크로 들어가서 본문을 크롤링
def get_content(df):
    contents = []
    # 웹 서버가 요청을 봇이 아닌 일반 사용자라고 판단하도록 User-Agent를 설정
    headers = {"User-Agent" : 
               "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    
    bar = tqdm(df['Link'], total = len(df['Link']))
    # 링크별로 크롤링
    for link in bar:
        bar.set_description(f'Porcessiong Stock {df.query("Link==@link").iloc[0,0]}')
        
        response = requests.get(link, headers=headers)
        bs_obj = BeautifulSoup(response.text, 'html.parser')
        tag_remover = bs_obj.select_one('div.link_news') # 제거하기 위한 태그 수집
        # 해당 링크에서는 불필요한 태그가 같이 수집되기 때문에 해당 태그를 제거
        if tag_remover:
            tag_remover.decompose()
        
        # 기사 본문을 크롤링해서 contents 리스트에 추가하며 content가 없거나 해당 코드에서 오류가 나면  공백을 추가
        try:
            content = bs_obj.select('#news_read')[0].text
            if content: contents.append(content)
            else: contents.append('')
        except:
            contents.append('')
            
        time.sleep(0.5)
    bar.close()
    
    # 데이터 프레임에 추가
    df['Content'] = contents
    
    return df

In [ ]:
# 각 데이터에 대해서 기사 본문 을 수집해서 새로운 데이터 프레임 생성
df_semiconductor_content = get_content(df_semiconductor)
df_electric_car_battery_content = get_content(df_electric_car_battery)
df_petrochemistry_content = get_content(df_petrochemistry)
df_shipbuilding_shiping_content = get_content(df_shipbuilding_shiping)
df_steel_content = get_content(df_steel)

In [ ]:
# 수집한 데이터 csv로 저장
df_semiconductor_content.to_csv('Title_Link_Date/df_semiconductor_content.csv', index=False)
df_electric_car_battery_content.to_csv('Title_Link_Date/df_electric_car_battery_content.csv', index=False)
df_petrochemistry_content.to_csv('Title_Link_Date/df_petrochemistry_content.csv', index=False)
df_shipbuilding_shiping_content.to_csv('Title_Link_Date/df_shipbuilding_shiping_content.csv', index=False)
df_steel_content.to_csv('Title_Link_Date/df_steel_content.csv', index=False)

In [17]:
display(df_semiconductor_content, df_electric_car_battery_content,
        df_petrochemistry_content, df_shipbuilding_shiping_content, df_steel_content)

,Stock_Item,Title,Link,Date,Content
0,삼성전자,[유안타證 주간추천주]삼성전자·두산·POSCO홀딩스,https://finance.naver.com/item/news_read.naver...,2023-06-04,"\n\t\t\t [이데일리 이정현 기자] △삼성전자(005930)-1분기 매출액, ..."
1,삼성전자,“5만원짜리 17만원에 샀어요” 삼성 ‘이 제품’ 뭐길래 못 사서 난리...,https://finance.naver.com/item/news_read.naver...,2023-06-03,\n삼성전자가 2년여 전 출시한 스마트밴드 ‘갤럭시 핏2’ [Mike O'Brien...
2,삼성전자,이러다 현대차도 제칠라...‘베트남의 삼성’이 아메리칸 드림 꿈 꿔보지...,https://finance.naver.com/item/news_read.naver...,2023-06-03,\n 빈패스트의 전기차 세단 VF8. <빈패스트 홈페이지>[신짜오 베트남-248]베...
3,삼성전자,“6월 삼성전자·카카오 뜬다”...반도체·인터넷 업종 강세 전망,https://finance.naver.com/item/news_read.naver...,2023-06-03,\n현대차증권 6월 월간 보고서 발간 삼성전자 주가는 최근 7만원을 돌파하며 강세를...
4,삼성전자,“이래도 애플페이 써야 돼?” 수수료 중국 5배 폭리…오히려 무료 결단...,https://finance.naver.com/item/news_read.naver...,2023-06-03,\n현대카드의 애플페이 광고[헤럴드경제= 박영훈기자] “우리나라는 글로벌 호구다”“...
...,...,...,...,...,...
13351,SK하이닉스,[신년사] 박정호 SK하이닉스 대표 “車·AI 분야서 신성장 동력 만...,https://finance.naver.com/item/news_read.naver...,2023-01-02,\n박정호 SKT 부회장이 지난해 11월 8일 서울 광진구 워커힐 호텔에서 열린 ‘...
13352,SK하이닉스,"[신년사] 박정호 SK하이닉스 부회장 ""원팀으로 위기 극복""",https://finance.naver.com/item/news_read.naver...,2023-01-02,\nSK하이닉스[연합뉴스TV 제공] (서울=연합뉴스) 김기훈 기자 = SK하이...
13353,SK하이닉스,박정호 SK하이닉스 대표 “도전 DNA로 위기 극복하자”,https://finance.naver.com/item/news_read.naver...,2023-01-02,\n박정호 SK하이닉스 부회장 신년 인사박정호 SK하이닉스 대표이사(부회장)[헤럴드...
13354,SK하이닉스,"[반도체 코리아]② SK하이닉스, 차세대 메모리 이끌 ‘만능열쇠 CXL...",https://finance.naver.com/item/news_read.naver...,2023-01-02,"\n강욱성 SK하이닉스 차세대상품기획 담당 부사장 “CXL로 메모리 설계, 종류, ..."


,Stock_Item,Title,Link,Date,Content
0,현대차,이러다 현대차도 제칠라...‘베트남의 삼성’이 아메리칸 드림 꿈 꿔보지...,https://finance.naver.com/item/news_read.naver...,2023-06-03,\n 빈패스트의 전기차 세단 VF8. <빈패스트 홈페이지>[신짜오 베트남-248]베...
1,현대차,"美서 돈 덜쓰고 더 많이 판 현대차…""제품력 개선, 펀더멘털 달라졌다""",https://finance.naver.com/item/news_read.naver...,2023-06-03,"\nNH투자증권 ""투자의견 긍정 유지""5월 美 빅3 스텔란티스에 또 앞서보조금은 1..."
2,현대차,“중고차 시장에선 내가 잘나가”…현대 아반떼·미니 컨트리맨 주목,https://finance.naver.com/item/news_read.naver...,2023-06-03,\n1000만원 초·중반대 인기 국산차는 아반떼수입차는 브랜드 특성 뚜렷한 차량 선...
3,현대차,"현대차 노조, ""40년 근속자에 금메달 달라"" 外[금주의 산업계 이슈]",https://finance.naver.com/item/news_read.naver...,2023-06-03,\n[울산=뉴시스]현대자동차 노조가 올해 임금협상 난항을 이유로 1일 파업 찬반투표...
4,현대차,현대차는 ‘세단’·기아는 ‘SUV’…1~5월 완성차 효자는?,https://finance.naver.com/item/news_read.naver...,2023-06-03,"\n현대차 5월 ‘그랜저’·‘아반떼’ 등 세단 강세기아, ‘카니발’·‘스포티지’ 등..."
...,...,...,...,...,...
11236,LG에너지솔루션,"[신년사] 권영수 LG엔솔 부회장 ""강한 실행력으로 더 큰 미래 준비...",https://finance.naver.com/item/news_read.naver...,2023-01-02,\nIT시스템 고도화·명확한 역할과 책임·팀워크 확립 3대 과제(서울=연합뉴스) 장...
11237,LG에너지솔루션,"""LG엔솔, 4분기 실적 컨센서스 하회 예상…'잠시 숨 고르기'""",https://finance.naver.com/item/news_read.naver...,2023-01-02,\n[한국투자증권 보고서]4분기 영업이익 컨센서스 밑돌 전망다만 투자의견 '매수'와...
11238,LG에너지솔루션,"LG엔솔, 4Q 실적 컨센서스 하회 '숨고르기'-한국",https://finance.naver.com/item/news_read.naver...,2023-01-02,\n\t\t\t [이데일리 양지윤 기자] 한국투자증권은 2일 삼성증권에 대해 작년 ...
11239,LG에너지솔루션,제2의 LG엔솔 나올까…“새해 IPO 침체 지속 전망”,https://finance.naver.com/item/news_read.naver...,2023-01-02,\n국내 10대 증권사 리서치센터장 2023년 증시전망기업 자금조달 환경 부진…IP...


,Stock_Item,Title,Link,Date,Content
0,LG화학,"LG화학, 450만 달러 투자한 기술 반환…항암제 역량 집중 시동",https://finance.naver.com/item/news_read.naver...,2023-06-02,\n스웨덴 스프린트 NASH 후보물질 최근 반환 진단사업부문 우선협상 대상자 선정...
1,LG화학,"신학철 LG화학 부회장, 일본서 ‘인재 확보’",https://finance.naver.com/item/news_read.naver...,2023-06-02,\n3대 신성장 동력 분야 인재 확보 위해 채용 행사 주관신학철 LG화학 부회장이 ...
2,LG화학,신학철 “LG화학 일원 돼 달라” 日 인재들에 러브콜,https://finance.naver.com/item/news_read.naver...,2023-06-02,\n3대 신성장동력 키울 인재 찾아 일본행신학철 LG화학 부회장이 일본 인터컨티넨털...
3,LG화학,"""LG화학에서 꿈 펼치라""…신학철 부회장, 日 인재에 '손짓'",https://finance.naver.com/item/news_read.naver...,2023-06-02,\n日 7개대 이공계 학생 대상 채용 행사소재·바이오 등 분야 인재 확보 목적[서울...
4,LG화학,"""인재 확보"" 신학철 LG화학 부회장, 일본까지 날아갔다",https://finance.naver.com/item/news_read.naver...,2023-06-02,\n전지소재·친환경소재·혁신신약 인재 확보글로벌 인재 확보 활동 'BC 투어' 진행...
...,...,...,...,...,...
4383,S-OIL,"에쓰오일, CES서 친환경 사업·벤처기업 등 소개",https://finance.naver.com/item/news_read.naver...,2023-01-06,\n기사내용 요약류열 전략관리총괄 사장 등 참가[서울=뉴시스] 류열 에쓰오일 전략관...
4384,S-OIL,"[CES 2023] 에쓰오일 ""미래 청정에너지 공급자로서 전략 고민""",https://finance.naver.com/item/news_read.naver...,2023-01-06,\n류열 에쓰오일 사장(오른쪽 2번째)이 5일(현지시간) CES 2023에서 에쓰오...
4385,S-OIL,"류열 에쓰오일 사장, ‘CES 2023’ 현장 참관…“전략 고민 기회...",https://finance.naver.com/item/news_read.naver...,2023-01-06,\n에쓰오일 경영진으로선 첫 참가…‘성장 방향성’ 점검‘직접 투자’ 리베스트·범준E...
4386,S-OIL,"류열 S-OIL 사장, CES 참관…신사업 분야 투자기업 격려",https://finance.naver.com/item/news_read.naver...,2023-01-06,\n◆…류열 S-OIL 사장(오른쪽 2번째)이 5일 미국 라스베이거스에서 열린 CE...


,Stock_Item,Title,Link,Date,Content
0,HD현대중공업,[유가증권시장 공시] HD현대중공업 / HSD엔진 / 한신공영 등,https://finance.naver.com/item/news_read.naver...,2023-06-01,\n\t\t\t ◇HD현대중공업=1조2392억원 규모 컨테이너선 5척 수주.◇HSD...
1,HD현대중공업,"HD현대중공업, 1조2392억 규모 컨테이너선 5척 수주[주목 e공시]",https://finance.naver.com/item/news_read.naver...,2023-06-01,\nHD현대중공업이 건조한 컨테이너선 / 사진=연합뉴스HD현대중공업은 1조2392억...
2,HD현대중공업,"[공시] HD현대중공업, 1조2400억 규모 컨테이너선 공사 수주",https://finance.naver.com/item/news_read.naver...,2023-06-01,\n\t\t\t [파이낸셜뉴스] HD현대중공업은 양밍해운(YANG MING MA...
3,HD현대중공업,"HD현대중공업, 1조2392억원 규모 컨테이너선 공사 수주",https://finance.naver.com/item/news_read.naver...,2023-06-01,\n\t\t\t [이데일리 김진호 기자]HD현대중공업(329180)이 ‘양 밍 마린...
4,HD현대중공업,"HD현대중공업, 1조2392억 규모 컨테이너선 5척 수주 계약",https://finance.naver.com/item/news_read.naver...,2023-06-01,\n\t\t\t [헤럴드경제=증권부] HD한국조선해양은 자회사 HD현대중공업이 양밍...
...,...,...,...,...,...
4122,팬오션,"'벌크선 강자' 팬오션, 대형 LNG선 첫 투입 '영토확장'",https://finance.naver.com/item/news_read.naver...,2023-01-05,\n\t\t\t 하림그룹의 해운 계열사이자 국내 최대 벌크선사인 팬오션이 5일 대형...
4123,팬오션,"팬오션, 글로벌 LNG시장 진출..첫 운반선 명명식 개최",https://finance.naver.com/item/news_read.naver...,2023-01-05,\n17.4만CBM급 운반선..'뉴에이펙스'로 명명포르투칼 GALP 장기대선계약에 ...
4124,팬오션,"팬오션, 글로벌 LNG 시장 진출…첫 운반선 '뉴에이펙스' 인도",https://finance.naver.com/item/news_read.naver...,2023-01-05,\n/사진제공=팬오션.팬오션은 삼성중공업 거제조선소에서 17만4000CBM급 LNG...
4125,팬오션,"'벌크선 강자' 팬오션, 대형 LNG운반선 첫 투입",https://finance.naver.com/item/news_read.naver...,2023-01-05,\n\t\t\t 하림그룹의 해운 계열사인 팬오션이 올해부터 대형 액화천연가스(LNG...


,Stock_Item,Title,Link,Date,Content
0,현대제철,"현대제철, 창립 70주년 걸음기부 캠페인…1억원 기금 조성 목표",https://finance.naver.com/item/news_read.naver...,2023-06-02,"\n◆…현대제철이 걸음기부 캠페인 '70cm의 기적, 문샷 챌린지'를 진행한다. 사..."
1,현대제철,"현대제철, 아동 이동체어 걸음 기부 캠페인 펼쳐",https://finance.naver.com/item/news_read.naver...,2023-06-02,\n\t\t\t 현대제철이 창립 70주년을 맞아 걸음 기부 캠페인을 펼친다.현대제철...
2,현대제철,'하루 5000보의 나눔'..창립 70년 현대제철 걸음기부 캠페인,https://finance.naver.com/item/news_read.naver...,2023-06-02,"\n'70cm의 기적, 문샷 챌린지'10일 창립기념일까지 22일간""목표 5억보"" 기..."
3,현대제철,"현대제철, '걸음 기부' 문샷 챌린지 진행…1억원 기부 목표",https://finance.naver.com/item/news_read.naver...,2023-06-02,"\n현대제철이 창립 70주년을 맞아 '70㎝의 기적, 문샷 챌린지(Moonshot ..."
4,현대제철,"현대제철, ‘걸음 기부 캠페인’ 문샷 챌린지 진행…1억원 기부 목표",https://finance.naver.com/item/news_read.naver...,2023-06-02,\n달까지의 거리인 38만km 나눠 걸으며 기부5000명이 일 5000걸음씩 22일...
...,...,...,...,...,...
6766,고려아연,"고려아연, 신사업 지배력 보여줘야할 때…투자의견 '매수'-유안타",https://finance.naver.com/item/news_read.naver...,2023-01-04,"\n유안타증권이 고려아연에 대한 투자의견을 '매수', 목표주가를 69만원으로 유지했..."
6767,고려아연,고려아연 차지하려는 '두 가문'…3월 주총서 '진검승부' 예상 [김익환...,https://finance.naver.com/item/news_read.naver...,2023-01-02,\n최윤범 일가 VS 장형진 일가고려아연 놓고 격돌 지분경쟁3월 주총서 이사진 절반...
6776,고려아연,"최씨, 고려아연 지분 경쟁 키 쥔 ‘영풍정밀’ 주식 매수",https://finance.naver.com/item/news_read.naver...,2023-01-04,\n고려아연 온산제련소. /고려아연 제공 영풍그룹 최씨 일가와 장씨 일...
6777,고려아연,"고려아연, 신사업 지배력 보여줘야할 때…투자의견 '매수'-유안타",https://finance.naver.com/item/news_read.naver...,2023-01-04,"\n유안타증권이 고려아연에 대한 투자의견을 '매수', 목표주가를 69만원으로 유지했..."
